# HOW TO RUN THIS FILE
1. edit the ROOT_PATH variable to match the path from your google drive to the eva shared folder
2. make sure that the runtime type is set to GPU. the code requires CUDA to work. (you can set it to CPU only mode, but it's about 10x slower)

In [1]:
!nvidia-smi

Mon Dec  6 00:44:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import sys
# EDIT THIS VARIABLE:
ROOT_PATH = "/content/drive/MyDrive/CS245 Project/CS245 Project Shared/eva"
SRC_PATH = os.path.join(ROOT_PATH, "src")
sys.path.append(SRC_PATH)
from run_dbp15k import *

In [4]:
def run_eva(emb_sizes=(100,100,200), 
            gcn_emb_size=200, 
            verbose=True, 
            save_emb=False,
            img_features=None):
    parser = argparse.ArgumentParser()
    parser.add_argument("--file_dir", type=str, default="data/DBP15K/zh_en", required=False, help="input dataset file directory, ('data/DBP15K/zh_en', 'data/DWY100K/dbp_wd')")
    parser.add_argument("--rate", type=float, default=0.3, help="training set rate")
    
    parser.add_argument("--cuda", action="store_true", default=True, help="whether to use cuda or not")
    parser.add_argument("--seed", type=int, default=2021, help="random seed")
    parser.add_argument("--epochs", type=int, default=1000, help="number of epochs to train")
    parser.add_argument("--check_point", type=int, default=100, help="check point")
    parser.add_argument("--hidden_units", type=str, default="128,128,128", help="hidden units in each hidden layer(including in_dim and out_dim), splitted with comma")
    parser.add_argument("--heads", type=str, default="2,2", help="heads in each gat layer, splitted with comma")
    parser.add_argument("--instance_normalization", action="store_true", default=False, help="enable instance normalization")
    parser.add_argument("--lr", type=float, default=0.005, help="initial learning rate")
    parser.add_argument("--weight_decay", type=float, default=0, help="weight decay (L2 loss on parameters)")
    parser.add_argument("--dropout", type=float, default=0.0, help="dropout rate for layers")
    parser.add_argument("--attn_dropout", type=float, default=0.0, help="dropout rate for gat layers")
    parser.add_argument("--dist", type=int, default=2, help="L1 distance or L2 distance. ('1', '2')")
    parser.add_argument("--csls", action="store_true", default=False, help="use CSLS for inference")
    parser.add_argument("--csls_k", type=int, default=10, help="top k for csls")
    parser.add_argument("--il", action="store_true", default=False, help="Iterative learning?")
    parser.add_argument("--semi_learn_step", type=int, default=10, help="If IL, what's the update step?")
    parser.add_argument("--il_start", type=int, default=500, help="If Il, when to start?")
    parser.add_argument("--bsize", type=int, default=7500, help="batch size")
    parser.add_argument("--unsup", action="store_true", default=False)
    parser.add_argument("--unsup_k", type=int, default=1000, help="|visual seed|")
    #parser.add_argument("--long_tail_analysis", action="store_true", default=False)
    parser.add_argument("--lta_split", type=int, default=0, help="split in {0,1,2,3,|splits|-1}")
    args = argparse.Namespace( 
        file_dir=os.path.join(ROOT_PATH, "data/DBP15K/fr_en"), 
        rate=0.3, 
        lr=0.0005, 
        epochs=50, #1000 originally
        hidden_units=f"400,400,{gcn_emb_size}", 
        check_point=50, 
        bsize=7500, 
        il=False, # True originally 
        il_start=500, 
        semi_learn_step=5, 
        csls=True, 
        csls_k=3, 
        seed=0,
    )
    args = parser.parse_args(args=[], namespace=args)
    results = main(
        args, 
        img_features=img_features,
        emb_sizes=emb_sizes, 
        save_emb=save_emb, 
        root_path=ROOT_PATH,
        verbose=verbose
    )
    return results

In [5]:
# default run:
results = run_eva(verbose=False)

loading raw data...
70.69% entities have images
image feature shape: torch.Size([39654, 2048])
#left entity : 19661, #right entity: 19993
#left entity not in train set: 15161, #right entity not in train set: 15493
relation feature shape: torch.Size([39654, 1000])
attribute feature shape: torch.Size([39654, 1000])
-----dataset summary-----
dataset:	 /content/drive/MyDrive/CS245 Project/CS245 Project Shared/eva/data/DBP15K/fr_en
triple num:	 221720
entity num:	 39654
relation num:	 2111
train ill num:	 4500 	test ill num:	 10500
-------------------------
getting a sparse tensor r_adj...
GCN model details:
GCN(
  (gc1): GraphConvolution (400 -> 400)
  (gc2): GraphConvolution (400 -> 200)
)
optimiser details:
AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0005
    weight_decay: 0.01
)
[start training...] 

[epoch 49] checkpoint!
normalised weights: tensor([0.2526, 0.2475, 0.2466, 0.2533], device='cuda:0')
l2r: acc of top [1, 10, 50] = [0.1855 

In [6]:
acc_l2r, acc_r2l, mean_l2r, mean_r2l, mrr_l2r, mrr_r2l = results
print(acc_l2r)
print(acc_r2l)
print(mean_l2r)
print(mean_r2l)
print(mrr_l2r)
print(mrr_r2l)

[0.1855 0.4508 0.6621]
[0.1873 0.4482 0.6607]
280.7753333333333
276.51819047619045
0.27278504433051803
0.2739942090124422


# EfficientNet Encodings

Using the image features extracted using EfficientNet B0/B7, we will compare it against the original ResNet152 encodings.

In [7]:
# extract list of ids
import pickle
data_dir = os.path.join(ROOT_PATH, "data")
FILE_DIR = os.path.join(ROOT_PATH, "data/DBP15K/fr_en")
ent2id_dict, _, _, _, _, _ = read_raw_data(FILE_DIR, [1,2])
with open(os.path.join(data_dir, "en_labels.pkl"), "rb") as f:
    en_labels = pickle.load(f)
with open(os.path.join(data_dir, "fr_labels.pkl"), "rb") as f:
    fr_labels = pickle.load(f)
all_labels = en_labels + fr_labels
all_ids = [ent2id_dict[label] for label in all_labels]

loading raw data...


In [8]:
# load image features
effnet_b0_features = np.load(os.path.join(ROOT_PATH, "data", "effnet_b0_imgfeatures.npy"))
effnet_b7_features = np.load(os.path.join(ROOT_PATH, "data", "effnet_b7_imgfeatures.npy"))

In [9]:
num_entities = len(ent2id_dict)
def pad_features(features):
    mean = np.mean(features, axis=0)
    std = np.std(features, axis=0)
    img_embd = np.array(
        [features[all_ids.index(i)] if i in all_ids 
         else np.random.normal(mean, std, mean.shape[0]) 
         for i in range(num_entities)])
    return img_embd

In [11]:
results_b0 = run_eva(verbose=False, img_features=pad_features(effnet_b0_features))
results_b7 = run_eva(verbose=False, img_features=pad_features(effnet_b7_features))

loading raw data...
image feature shape: torch.Size([39654, 1280])
#left entity : 19661, #right entity: 19993
#left entity not in train set: 15161, #right entity not in train set: 15493
relation feature shape: torch.Size([39654, 1000])
attribute feature shape: torch.Size([39654, 1000])
-----dataset summary-----
dataset:	 /content/drive/MyDrive/CS245 Project/CS245 Project Shared/eva/data/DBP15K/fr_en
triple num:	 221720
entity num:	 39654
relation num:	 2111
train ill num:	 4500 	test ill num:	 10500
-------------------------
getting a sparse tensor r_adj...
GCN model details:
GCN(
  (gc1): GraphConvolution (400 -> 400)
  (gc2): GraphConvolution (400 -> 200)
)
optimiser details:
AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0005
    weight_decay: 0.01
)
[start training...] 

[epoch 49] checkpoint!
normalised weights: tensor([0.2527, 0.2475, 0.2465, 0.2532], device='cuda:0')
l2r: acc of top [1, 10, 50] = [0.2495 0.4887 0.6848], mr = 263.872

In [12]:
import pickle
with open(os.path.join(ROOT_PATH, "results_effnet_b0.pkl"), "wb") as f:
    pickle.dump(results_b0, f)
with open(os.path.join(ROOT_PATH, "results_effnet_b7.pkl"), "wb") as f:
    pickle.dump(results_b7, f)